In [1]:
# CELL 1B: VERIFY INSTALLS
try:
    import pdf2image
    print("✅ pdf2image OK")
except:
    print("❌ pdf2image - install poppler")

try:    
    import pytesseract
    print("✅ pytesseract OK")
except:
    print("❌ pytesseract - install Tesseract")

try:
    import easyocr
    print("✅ easyocr OK")
except:
    print("❌ easyocr")

try:
    import paddleocr
    print("✅ paddleocr OK")
except:
    print("❌ paddleocr")

try:
    import cv2
    print("✅ opencv OK")
except:
    print("❌ opencv")

print("\n🚀 Ready if all ✅ above!")


✅ pdf2image OK
✅ pytesseract OK



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\heman\anaconda3\envs\Realtime\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\heman\anaconda3\envs\Realtime\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\heman\anaconda3\envs\Realtime\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  Fi

✅ easyocr OK


Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


❌ paddleocr
✅ opencv OK

🚀 Ready if all ✅ above!


In [3]:
# CELL 1D: PADDLEOCR FIX + FINAL TEST (Run This)
import os
os.environ['PADDLEOCR_DISABLE_MODEL_SOURCE_CHECK'] = 'True'  # Skip model check
PDF_PATH=r"C:\Drive_d\Python\F-AI\T4\Input\225790297864463.pdf"
# Test PaddleOCR separately
try:
    from paddleocr import PaddleOCR
    ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False, use_gpu=False)
    result = ocr.ocr(PDF_PATH, cls=True)
    print("✅ paddleocr OK (tested on your PDF!)")
except Exception as e:
    print(f"❌ paddleocr: {e}")
    print("💡 Fix: Methods work WITHOUT paddleocr too!")

# Full PDF test
from pdf2image import convert_from_path
images = convert_from_path(PDF_PATH, first_page=1, last_page=1, dpi=100)
print(f"🎯 PDF: {len(images)} page(s) loaded OK")

print("\n🚀 ALL SYSTEMS GO! Run Cells 2-6 now.")


❌ paddleocr: PDX has already been initialized. Reinitialization is not supported.
💡 Fix: Methods work WITHOUT paddleocr too!
🎯 PDF: 1 page(s) loaded OK

🚀 ALL SYSTEMS GO! Run Cells 2-6 now.


In [4]:
def save_result(method_name: str, ocr_text: str, extra_info: dict = None):
    """Save OCR output to file"""
    filename = "225790297864463_" + method_name + ".txt"
    out_path = os.path.join(r"C:\Drive_d\Python\F-AI\T4\Output", filename)
    
    content = f"=== {method_name.upper()} ===\n"
    if extra_info:
        content += f"Info: {extra_info}\n\n"
    content += ocr_text
    
    with open(out_path, 'w', encoding='utf-8') as f:
        f.write(content)
    
    line_count = len([l for l in ocr_text.splitlines() if l.strip()])
    print(f"💾 {method_name}: {line_count} lines → {out_path}")
    return out_path

print("✅ Helpers ready!")


✅ Helpers ready!


In [10]:
import numpy as np
# METHOD 1: SIMPLE VOTING ENSEMBLE (FIXED)
print("\n🔍 METHOD 1: SIMPLE VOTING ENSEMBLE")
print("=" * 50)

PDF_PATH = r"C:\Drive_d\Python\F-AI\T4\Input\225790297864463.pdf"
images = convert_from_path(PDF_PATH, dpi=200, first_page=1, last_page=1)  # 1 page

img = images[0]  # PIL Image

# Tesseract (if PATH fixed)
try:
    tess_text = pytesseract.image_to_string(img)
except:
    tess_text = "TESSERACT UNAVAILABLE"

# EasyOCR (convert PIL → numpy)
img_np = np.array(img)
reader = easyocr.Reader(['en'], gpu=False)
easy_results = reader.readtext(img_np, detail=0)
easy_text = '\n'.join(easy_results)

# PaddleOCR (convert PIL → numpy)
try:
    paddle_ocr = paddleocr.PaddleOCR(show_log=False, use_gpu=False)
    paddle_results = paddle_ocr.ocr(img_np, cls=True)
    paddle_text = '\n'.join([line[1][0] for page in paddle_results for line in page])
except:
    paddle_text = easy_text

# Simple voting
texts = [tess_text, easy_text, paddle_text]
all_lines = []
for t in texts:
    all_lines.extend([line.strip() for line in t.split('\n') if line.strip()])

line_votes = Counter(all_lines)
top_lines = [line for line, count in line_votes.most_common(50)]
final_text = '\n'.join(top_lines)

save_result("method1_voting_fixed", final_text)
print("✅ Method 1 FIXED & COMPLETE!")
print("Preview:\n", final_text[:300])



🔍 METHOD 1: SIMPLE VOTING ENSEMBLE


Using CPU. Note: This module is much faster with a GPU.


RuntimeError: Numpy is not available